In [1]:
# Here we are just importing some libraries which are needed to run this notebook.
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage

from denoiseg.models import DenoiSeg, DenoiSegConfig
from denoiseg.utils.misc_utils import combine_train_test_data, shuffle_train_data, augment_data
from denoiseg.utils.seg_utils import *
from denoiseg.utils.compute_precision_threshold import measure_precision

from csbdeep.utils import plot_history

import tensorflow as tf
import keras.backend as K

import urllib
import os
import zipfile

Using TensorFlow backend.
/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/random/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/random/.local/lib/python3.6/site-packages/tensorflow/python

In [2]:
X = np.random.uniform(-2,2,(10, 96, 96, 1))
X_val = np.random.uniform(-2,2,(10, 96, 96, 1))
Y = np.random.uniform(-2,2,(10, 96, 96, 3))
Y_val = np.random.uniform(-2,2,(10, 96, 96, 3))

In [3]:
train_batch_size = 128
train_steps_per_epoch = max(100, min(int(X.shape[0]/train_batch_size), 400))

In [4]:
conf = DenoiSegConfig(X, unet_kern_size=3, n_channel_out=4, relative_weights = [1.0,1.0,5.0],
                      train_steps_per_epoch=train_steps_per_epoch, train_epochs=200, 
                      batch_norm=True, train_batch_size=128, unet_n_first = 32, 
                      unet_n_depth=4, denoiseg_alpha=0.5, train_tensorboard=False)

vars(conf)

{'means': ['-0.0008584833187920531'],
 'stds': ['1.155310532544392'],
 'n_dim': 2,
 'axes': 'YXC',
 'n_channel_in': 1,
 'n_channel_out': 4,
 'train_loss': 'denoiseg',
 'unet_n_depth': 4,
 'relative_weights': [1.0, 1.0, 5.0],
 'unet_kern_size': 3,
 'unet_n_first': 32,
 'unet_last_activation': 'linear',
 'probabilistic': False,
 'unet_residual': False,
 'unet_input_shape': (None, None, 1),
 'train_epochs': 200,
 'train_steps_per_epoch': 100,
 'train_learning_rate': 0.0004,
 'train_batch_size': 128,
 'train_tensorboard': False,
 'train_checkpoint': 'weights_best.h5',
 'train_checkpoint_last': 'weights_last.h5',
 'train_checkpoint_epoch': 'weights_now.h5',
 'train_reduce_lr': {'monitor': 'val_loss', 'factor': 0.5, 'patience': 10},
 'batch_norm': True,
 'n2v_perc_pix': 1.5,
 'n2v_patch_shape': (64, 64),
 'n2v_manipulator': 'uniform_withCP',
 'n2v_neighborhood_radius': 5,
 'denoiseg_alpha': 0.5}

# <div style='color:red'>Tensorboard is not working!</div>

In [5]:
model_name = 'denoiseg_model'
basedir = 'models'
model = DenoiSeg(conf, model_name, basedir)

Instructions for updating:
Colocations handled automatically by placer.


/home/random/miniconda3/envs/denoiseg/lib/python3.6/site-packages/denoiseg/models/denoiseg_standard.py:533: UserWarning: output path for model already exists, files may be overwritten: /media/data/Development/imagej/project/CSBDeep/DenoiSeg_fiji/src/main/python/models/denoiseg_model
  'output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


In [6]:
model.prepare_for_training()
model.keras_model._make_train_function()
model.keras_model._make_test_function()
model.keras_model._make_predict_function()
#model.prepare_for_training()
model.keras_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, None, None, 1 0                                            
__________________________________________________________________________________________________
down_level_0_no_0 (Conv2D)      (None, None, None, 3 320         input[0][0]                      
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         down_level_0_no_0[0][0]          
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [7]:
nodes = [n.name for n in tf.get_default_graph().as_graph_def().node]
[k for k in nodes if 'group' in k]

['training/group_deps', 'group_deps', 'group_deps_1']

In [8]:
#loss = tf.get_default_graph().get_tensor_by_name("loss/mul:0")
#train_op = tfoptimizer.minimize(loss, name='train')
tf.global_variables_initializer()
tf.train.Saver().as_saver_def()

filename_tensor_name: "save/Const:0"
save_tensor_name: "save/control_dependency:0"
restore_op_name: "save/restore_all"
max_to_keep: 5
keep_checkpoint_every_n_hours: 10000.0
version: V2

In [9]:
tf.summary.FileWriter("logs", tf.get_default_graph())

In [10]:
with open('../resources/graph_2d.pb', 'wb') as f:
    f.write(tf.get_default_graph().as_graph_def().SerializeToString())

In [11]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['input',
 'down_level_0_no_0/random_uniform/shape',
 'down_level_0_no_0/random_uniform/min',
 'down_level_0_no_0/random_uniform/max',
 'down_level_0_no_0/random_uniform/RandomUniform',
 'down_level_0_no_0/random_uniform/sub',
 'down_level_0_no_0/random_uniform/mul',
 'down_level_0_no_0/random_uniform',
 'down_level_0_no_0/kernel',
 'down_level_0_no_0/kernel/Assign',
 'down_level_0_no_0/kernel/read',
 'down_level_0_no_0/Const',
 'down_level_0_no_0/bias',
 'down_level_0_no_0/bias/Assign',
 'down_level_0_no_0/bias/read',
 'down_level_0_no_0/convolution/dilation_rate',
 'down_level_0_no_0/convolution',
 'down_level_0_no_0/BiasAdd',
 'batch_normalization_1/Const',
 'batch_normalization_1/gamma',
 'batch_normalization_1/gamma/Assign',
 'batch_normalization_1/gamma/read',
 'batch_normalization_1/Const_1',
 'batch_normalization_1/beta',
 'batch_normalization_1/beta/Assign',
 'batch_normalization_1/beta/read',
 'batch_normalization_1/Const_2',
 'batch_normalization_1/moving_mean',
 'batch_norm

In [13]:
# save for prediction
import shutil
shutil.rmtree("../resources/prediction_2d")
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    tf.tables_initializer().run()
    #print(sess.graph.get_tensor_by_name('keras_learning_phase/input:0'))
    #sess.run(tf.variables_initializer([tf.compat.v1.get_variable("conv2d_1/bias:0")]))
    #sess.run(init)
    sess.run("init")
    model_input = tf.compat.v1.saved_model.build_tensor_info(sess.graph.get_tensor_by_name('input:0'))
    model_output = tf.compat.v1.saved_model.build_tensor_info(sess.graph.get_tensor_by_name('activation_19/Identity:0'))

    signature_definition = tf.compat.v1.saved_model.build_signature_def(
        inputs={"input": model_input},
        outputs={"output": model_output},
        method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME)

    builder = tf.compat.v1.saved_model.Builder("../resources/prediction_2d")
    builder.add_meta_graph_and_variables(sess, [tf.compat.v1.saved_model.tag_constants.SERVING], signature_def_map={
        tf.compat.v1.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_definition
    }, clear_devices=True)
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../resources/prediction_2d/saved_model.pb
